# import libraries

In [12]:
import os
import json
import re
import pandas as pd
from C_D_create_embeddings import embed_caption_api

# create database

In [14]:
# Load the CSV file into a DataFrame
captions_path = "C:/Users/Esra/Desktop/Deep_Learning/Image_Classification/Fashion/Classify_ThreadUp_Images/dedupped_descriptions3.csv"
captions = pd.read_csv(captions_path)

# Define a function to parse the descriptions
def parse_description(description):
    # Create a dictionary to hold the parsed data
    parsed_data = {
        'Specific Category': None,
        'Color': None,
        'Material': None,
        'Features': None,
        'Style': None,
        'Occasion': None
    }
    # Split the description into lines
    lines = description.split('\n')
    # Process each line
    for line in lines:
        # Use regex to capture the field name and value
        match = re.match(r'- (.*?): (.*)', line)
        if match:
            field_name = match.group(1).strip()
            field_value = match.group(2).strip()
            # Map the field name to the corresponding dictionary key
            if field_name in parsed_data and field_name:
                parsed_data[field_name] = field_value
    return pd.Series(parsed_data)

# Apply the function to each description and join the result to the original DataFrame
parsed_descriptions = captions['description'].apply(parse_description)
captions = captions.join(parsed_descriptions)

#drop descriptions column
# captions.drop('description', axis=1, inplace=True)

# drop the images with none values
captions = captions.dropna().reset_index(drop=True)
captions

,filename,description,Specific Category,Color,Material,Features,Style,Occasion
0,item100001657.jpg,- Specific Category: Trousers\n- Color: Solid ...,Trousers,"Solid coral red with a slightly glossy finish,...","Appears to be a synthetic blend, possibly with...","Front button closure with zipper fly, possibly...","Straight-leg fit with a relaxed cut, likely in...",Suitable for business casual events or elevate...
1,item100246366.jpg,- Specific Category: Blouse\n- Color: White wi...,Blouse,"White with a soft matte finish, no visible pat...","Appears to be a lightweight, woven fabric, pos...","Button-up front closure with white buttons, lo...",Fashion-forward feminine style with a structur...,Suitable for business
2,item103003446.jpg,- Specific Category: Blouse\n- Color: Deep plu...,Blouse,Deep plum with a solid pattern and a somewhat ...,"Appears to be a lightweight fabric, potentiall...","Long sleeves with slits and button cuffs, V-ne...","Relaxed fit with a flowing silhouette, potenti...","Suitable for business casual environments, sem..."
3,item103756367.jpg,- Specific Category: Blouse\n- Color: Deep bur...,Blouse,Deep burgundy with a slightly crinkled texture...,"Looks like a synthetic blend, possibly with vi...","Long sleeves, V-neckline without closures, no ...","Relaxed fit with a flowing silhouette, likely ...","Suitable for smart-casual occasions, office we..."
4,item104132211.jpg,- Specific Category: Blouse\n- Color: Off-whit...,Blouse,"Off-white or ivory, with no visible patterns a...","Appears to be a lightweight, possibly syntheti...","Classic shirt collar, front button closure, bu...","Tailored fit with long sleeves, intended to be...",Suitable for professional settings
5,item105067431.jpg,- Specific Category: Trousers\n- Color: Deep r...,Trousers,"Deep red with a slight sheen, uniform throughout","Appears to be a synthetic blend, possibly poly...","Flat front, no visible pockets or closures, an...","Straight leg cut with a slightly relaxed fit, ...",Suitable for business casual settings or smart...
6,item106068596.jpg,- Specific Category: Dress Shirt\n- Color: Whi...,Dress Shirt,"White with a semi-gloss finish, no patterns","Appears to be a smooth, lightweight cotton or ...","Button-up front, a standard pointed collar, lo...","Traditional, with a regular fit and a straight...",Suitable for formal events
7,item106129754.jpg,- Specific Category: Long-sleeve blouse\n- Col...,Long-sleeve blouse,"Deep purple with a slight sheen, uniform throu...","Appears to be a lightweight knit fabric, possi...","Round neckline, asymmetrical hemline that is l...","Loose, relaxed fit with a flowing drape, exten...",Suitable for casual to smart


# call embedding function

In [ ]:
# Assume captions is a pandas DataFrame with text data in each column

# Initialize a dictionary to store embeddings for each column
embeddings_dict = {
    'Specific Category': [],
    'Color': [],
    'Material': [],
    'Features': [],
    'Style': [],
    'Occasion': []
}

# Define the batch size
batch_size = 50

# Function to process a batch of texts and return their embeddings
def process_batch(text_list, subembed_key):
    batch_embeddings = embed_caption_api(text_list)
    embeddings_dict[subembed_key].extend(batch_embeddings)

# Loop through each pandas column (subembed_key) and get embeddings for each entry
for subembed_key in embeddings_dict.keys():
    text_list = []  # Initialize/reset the list for each column
    for text in captions[subembed_key]:
        # Clean the text and add it to the batch list
        text_list.append(text.replace("\n", " "))
        
        # Check if the batch size is reached or it's the end of the list
        if len(text_list) == batch_size:
            # Process the current batch
            process_batch(text_list, subembed_key)
            # Reset the list for the next batch
            text_list = []
    
    # Process any remaining texts in the last batch (if not empty)
    if text_list:
        process_batch(text_list, subembed_key)


In [ ]:
# # Initialize a dictionary to store embeddings for each column
# embeddings_dict = {
#     'Specific Category': [],
#     'Color': [],
#     'Material': [],
#     'Features': [],
#     'Style': [],
#     'Occasion': []
# }

# # Loop through each pandas column (subembed_key) and get embeddings for each entry
# for subembed_key in embeddings_dict.keys():
#     text_list = []
#     for text in captions[subembed_key]:
#         text_list.append(text.replace("\n", " "))
    
#     for 
#     embeddings_dict[subembed_key].append(embed_caption_api(text_list))
    

In [17]:
# Specify the filename
filename = 'embeddings_dict_batch.json'

with open(filename, 'w') as f:
    json.dump(embeddings_dict, f, indent=4)
    
#maybe in future, when you have a bigger database, json might be slower, use yaml instead